<a href="https://colab.research.google.com/github/osullik/ic23050/blob/main/DataDictionary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [360]:
from google.colab import drive
import pandas as pd
import json
from collections import defaultdict

In [361]:
DD = pd.ExcelFile("https://github.com/osullik/ic23050/blob/main/L4%20Washington%20Fatal%20Crash%20Files%20-%20Washington%20Traffic%20Safety%20Commission/DataDictionary_Washington%20Fatal%20Crash%20Files.xlsx?raw=true")

In [3]:
for sheet in DD.sheet_names:
  print(sheet)
  


ReadMe
CRASH Data Files
PERSON Data Files
Tables1
Tables2
Tables3
Tables4
Tables5
Tables6
Tables7
Tables8
Tables9


In [362]:
#Seperate out each sheet in the data dictionary 
ReadMe = pd.read_excel(DD, 'ReadMe')
CrashDataFiles = pd.read_excel(DD, 'CRASH Data Files')
PersonDataFiles = pd.read_excel(DD, 'PERSON Data Files')
#Tables1 = pd.read_excel(DD, 'Tables1')
#Tables2 = pd.read_excel(DD, 'Tables2')
#Tables3 = pd.read_excel(DD, 'Tables3')
#Tables4 = pd.read_excel(DD, 'Tables4')
#Tables5 = pd.read_excel(DD, 'Tables5')
#Tables6 = pd.read_excel(DD, 'Tables6')
#Tables7 = pd.read_excel(DD, 'Tables7')
#Tables8 = pd.read_excel(DD, 'Tables8')
Tables9 = pd.read_excel(DD, 'Tables9')

In [363]:
'''
getKVPairs is a function to extract KV Pairs from dirty excel data
  Input Args:
    desc: a String "description" of the values we want to extract. Should 
      include target KV pairs in the form KEY=VALUE where KEY is a number and 
      VALUE are non-numeric substrings. 
  Method: 
    Break the string into characters
    Accumulate numeric values until an "=" is seen. The "=" is an assignment  
      we infer the numbers prior are the key and this is appended to keyList
    Accumulate characters until either EOF or another numerical value is found
      we infer all strings between keys are their associated values. 
    
    Before appending a value to the definition list, remove newline and 
      truncate spaces

    Zip the KeyList and DefinitionList together, and return

  Returns:
    A list of Tuples in the form [(Key, Value)]

'''
def getKVPairs(desc):
                                                #Init Vars
  number = ""
  definition = ""
  keyList = []
  definitionList = []
  keyFound = False
  isComment = False
  isDefinition = False
                                                #Filter out non-strings
  if type(desc) == str:
    
    for char in desc:
     
      if char == ")":                           #Drops comments like "(Since 2010)"
        isComment = False
        continue
      elif char == "(":
        isComment = True
        continue

      if isComment == True:
        continue 
      else:
        if char.isnumeric() == True:  #Build the Keys

          if keyFound == True:
            definition = definition.strip("\n")
            definition = definition.strip()
            definitionList.append(definition)
            definition = ""
            keyFound = False
            number += char

          else:                                   #Build the Values
            number += char

        elif char == ("="):                       #Commit the Key
          keyList.append(number)
          number = ""
          keyFound = True

        else:                                     #Build the value
          definition += char 

    else:
      pass
      
                                                  #Add the last Definition
      definition = definition.strip("\n")
      definition = definition.strip()
      definitionList.append(definition)

  toReturn = list(zip(keyList,definitionList))  #Create a list of (K,V) Tuples

  return(toReturn)    

In [364]:
'''
getKVPairsSpaceDelimited is a function to extract KV Pairs from dirty excel data
  when there is a lack of delimiting "=" to seperate the Keys and Values
  Input Args:
    desc: a String "description" of the values we want to extract. Should 
      include target KV pairs in the form "KEY VALUE" where KEY is a number and 
      VALUE are non-numeric substrings. 
  Method: 
    Break the string into characters
    Accumulate numeric values until an " " is seen aftera number. 
      we infer the numbers prior to the space are the key and this is appended 
      to keyList
    Accumulate characters until either EOF or another numerical value is found
      we infer all strings between keys are their associated values. 
    
    Before appending a value to the definition list, remove newline and 
      truncate spaces

    Zip the KeyList and DefinitionList together, and return

  Returns:
    A list of Tuples in the form [(Key, Value)]

'''

def getKVPairsSpaceDelimited(desc):

                                                #Init Vars
  number = ""
  definition = ""
  keyList = []
  definitionList = []
  keyFound = False
  lastCharNumeric = False
                                                #Filter out non-strings
  if type(desc) == str:
    for char in desc:

      #print("Char=", char, "LastCharNumeric:",lastCharNumeric)
      
      if char.isnumeric() == True:              #Build the Keys
        if keyFound == True:
          definition = definition.strip("\n")
          definition = definition.strip()
          definitionList.append(definition)
          definition = ""
          keyFound = False
          number += char

        else:                                   #Build the Values
          number += char
          lastCharNumeric = True


      elif char == (" ") and lastCharNumeric == True and number != "":    #Commit the Key
        keyList.append(number)                                            
        number = ""
        keyFound = True
        lastCharNumeric == False

      else:                                     #Build the value
        definition += char 
    else:
      pass
                                                #Add the last Definition
    definition = definition.strip("\n")
    definition = definition.strip()
    definitionList.append(definition)

  toReturn = list(zip(keyList,definitionList))  #Create a list of (K,V) Tuples
  return(toReturn)    

In [16]:
#Parse the CrashDataFile into JSON
dataDict = defaultdict(dict)                    #Build the Data Dictionary

for idx, series in CrashDataFiles.iterrows():  #Step through each var in turn
  var = series['VARIABLE']                      #The loop extracts one record
  typ = series['TYPE']                          #And then selects relevant cells
  length = series['LENGTH']                     #The cells get passed to the 
  format = series['FORMAT']                     #getKVPairs() function below
  label = series['LABEL']
  desc = series['DESCRIPTION/COMMENT']

                                                #Populate the Data Dictionary
  dataDict[var]["TYPE"]     = typ
  dataDict[var]["LENGTH"]   = length 
  dataDict[var]["FORMAT"]   = format 
  dataDict[var]["LABEL"]    = label 

  kvPairList = getKVPairs(desc)                 #Get list (K,V) tuples for DATA

  dataDict[var]["DATA"]    = dict(kvPairList)

  


In [7]:
#Create JSON object out of CrashFileData
json_CrashDataFile = json.dumps(dataDict, indent = 4) 

In [8]:
#Parse the PersonDataFile into JSON
dataDict_PersonData = defaultdict(dict)                    #Build the Data Dictionary

print(PersonDataFiles.columns)

for idx, series in PersonDataFiles.iterrows():   #Step through each var in turn
  var = series['VARIABLE']
  typ = series['TYPE']
  length = series['LENGTH']
  format = series['FORMAT']
  label = series['LABEL']
  desc = series['DESCRIPTION/COMMENT']

                                                #Populate the Data Dictionary
  dataDict_PersonData[var]["TYPE"]     = typ
  dataDict_PersonData[var]["LENGTH"]   = length 
  dataDict_PersonData[var]["FORMAT"]   = format 
  dataDict_PersonData[var]["LABEL"]    = label 

  kvPairList = getKVPairs(desc)                 #Get list (K,V) tuples for DATA

  dataDict_PersonData[var]["DATA"]    = dict(kvPairList)

Index(['Unnamed: 0', 'VARIABLE', 'TYPE', 'LENGTH', 'FORMAT', 'LABEL',
       'DESCRIPTION/COMMENT', 'Notes / SAS Code', 'Unnamed: 8', 'Unnamed: 9'],
      dtype='object')


In [9]:
#Create JSON object out of PersonFileData
json_PersonDataFile = json.dumps(dataDict_PersonData, indent = 4) 

In [10]:
#Extract the messy table data from the TABLES1 Sheet 
  #(Manual inspection used to determine the cut points for the DFs)
Tables1_county = pd.read_excel(DD, 'Tables1', skiprows = 1, nrows=40,
                               usecols= 'A')
Tables1_harmfulEvent = pd.read_excel(DD, 'Tables1', skiprows = 1, nrows=40,  
                                     usecols= 'B,D')
Tables1_trafficControl = pd.read_excel(DD, 'Tables1', skiprows = 42, nrows=20,  
                                       usecols= 'A,D')
Tables1_seatingPosition = pd.read_excel(DD, 'Tables1', skiprows = 0, nrows=31,  
                                        usecols= 'E')


In [51]:
#Parse out the County Numbering System
dataDict_Table1 = defaultdict(dict)                    #Build the Data Dictionary

#print(Tables1_county.columns)
dataDict_Table1["County_Name"]["DATA"] = {}

for idx, series in Tables1_county.iterrows():   #Step through each var in turn
  
  desc = series['county_char']

  kvPairList = getKVPairs(desc)                 #Get list (K,V) tuples for DATA
  kvPairTuple = tuple(kvPairList[0])

  dataDict_Table1["County_Name"]["DATA"][kvPairTuple[0]] = kvPairTuple[1]

In [12]:
#Parse out the Harmful Event Data from Table1

dataDict_Table1["Harmful_Event"]["DATA"] = {}

for idx, series in Tables1_harmfulEvent.iterrows():   #Step through each var in turn
  
  desc1 = series['Non-Collision']
  desc2 = series['Unnamed: 3']

  kvPairList1 = getKVPairs(desc1)                 #Get list (K,V) tuples for DATA
  kvPairList2 = getKVPairs(desc2)


  #Following try/catch statements account for misalignment of columns in 
  #input data
  try:                                            #Transform into tuples
    kvPairTuple1 = tuple(kvPairList1[0])
  except IndexError:
    pass

  try:
    kvPairTuple2 = tuple(kvPairList2[0])
  except IndexError:
    pass

  try:                                           #Update the Dicts
    dataDict_Table1["Harmful_Event"]["DATA"][kvPairTuple1[0]] = kvPairTuple1[1]
  except NameError:
    pass
  try:
    dataDict_Table1["Harmful_Event"]["DATA"][kvPairTuple2[0]] = kvPairTuple2[1]
  except NameError:
    pass

In [13]:
#Parse out the Traffic Control Data from Table1

dataDict_Table1["Traffic_Control"]["DATA"] = {}

for idx, series in Tables1_trafficControl.iterrows():   #Step through each var in turn
  
  desc1 = series['TRAFFIC CONTROL DEVICE']
  desc2 = series['Unnamed: 3']

  kvPairList1 = getKVPairs(desc1)                 #Get list (K,V) tuples for DATA
  kvPairList2 = getKVPairs(desc2)

  #Following try/catch statements account for misalignment of columns in 
  #input data
  try:                                            #Transform into tuples
    kvPairTuple1 = tuple(kvPairList1[0])
  except IndexError:
    pass

  try:
    kvPairTuple2 = tuple(kvPairList2[0])
  except IndexError:
    pass

  try:                                           #Update the Dicts
    dataDict_Table1["Traffic_Control"]["DATA"][kvPairTuple1[0]] = kvPairTuple1[1]
  except NameError:
    pass
  try:
    dataDict_Table1["Traffic_Control"]["DATA"][kvPairTuple2[0]] = kvPairTuple2[1]
  except NameError:
    pass

In [14]:
#Parse out the Seating Positions
dataDict_Table1["Seating_Position"]["DATA"] = {}

for idx, series in Tables1_seatingPosition.iterrows():   #Step through each var in turn
  
  desc = series['SEATING POSITION']

  kvPairList = getKVPairs(desc)                 #Get list (K,V) tuples for DATA
  try:
    kvPairTuple = tuple(kvPairList[0])
    dataDict_Table1["Seating_Position"]["DATA"][kvPairTuple[0]] = kvPairTuple[1]
  except IndexError:
    pass

In [15]:
#Create a JSON object of the Table1 Data
json_Table1 = json.dumps(dataDict_Table1, indent = 4) 

In [121]:
#Extract the messy table data from the TABLES2 Sheet 
  #(Manual inspection used to determine the cut points for the DFs)
Tables2_driverViolations = pd.read_excel(DD, 'Tables2', skiprows = 0, nrows=5,
                               usecols= 'A')
Tables2_driverViolations = Tables2_driverViolations.append(pd.read_excel(DD, 
                              'Tables2', skiprows = 4, nrows=1, header=None,
                               usecols= 'B:D'))

Tables2_safetyRestraint = pd.read_excel(DD, 'Tables2', skiprows = 0, nrows = 3, 
                                        usecols = 'B')

Tables2_driverLicense = pd.read_excel(DD, 'Tables2', skiprows = 0, nrows = 3, 
                                        usecols = 'C,D')

In [122]:
def flattenLists(inputListofLists):

  #print(inputListofLists)

  flattenedList = []

  for sublist in inputListofLists:
    flattenedList.append(sublist)
    
  #print(flattenedList)
  return flattenedList

In [127]:
Tables2_driverViolations

,DRIVER VIOLATIONS 1-6,1,2,3
0,0=None\n1=Manslaughter or homicide \n2=Willful...,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,16=Driving with detectable alcohol \n18=Refusa...,NaN,NaN,NaN
3,41=Turn violation of traffic control (disobey ...,NaN,NaN,NaN
0,NaN,"58=Following too closely\n59=Wrong side, passi...",81=Lamp violation\n82=Brake violation\n83=Safe...,"\n87=Ice, Snow, Slush, Water, Sand, Dirt, Oil,..."


In [145]:
#Parse out the Driver Violations Numbering System
dataDict_Table2 = defaultdict(dict)                    #Build the Data Dictionary

#Tables2_driverViolations.columns
dataDict_Table2["Driver_Violations"]["DATA"] = {}

desc1 = None
desc2 = None
desc3 = None
desc4 = None

dv_flattenedList = []


for idx, series in Tables2_driverViolations.iterrows():   #Step through each var in turn
  
  desc1 = series['DRIVER VIOLATIONS 1-6']
  desc2 = series[1]
  desc3 = series[2]
  desc4 = series[3]

  dv_kvPairList1 = getKVPairs(desc1)                 #Get list (K,V) tuples for DATA
  dv_kvPairList2 = getKVPairs(desc2)                  #Var names changed to stop
  dv_kvPairList3 = getKVPairs(desc3)                  #X-contamination in Notebook
  dv_kvPairList4 = getKVPairs(desc4)

  dv_flattenedList += flattenLists(dv_kvPairList1)    #Flatten the lists into one
  dv_flattenedList += flattenLists(dv_kvPairList2)      #list of tuples
  dv_flattenedList += flattenLists(dv_kvPairList3)
  dv_flattenedList += flattenLists(dv_kvPairList4)


for tup in dv_flattenedList:                       #Populate the DataDict
  dataDict_Table2["Driver_Violations"]["DATA"][tup[0]] = tup[1]


In [142]:
#Parse out the Safety Restraint Data
dataDict_Table2["Safety_Restraint"]["DATA"] = {}
flattenedList = []

for idx, series in Tables2_safetyRestraint.iterrows(): 

  desc = series['SAFETY RESTRAINT USE']

  kvPairList = getKVPairs(desc)                 #Get list (K,V) tuples for DATA
  
  flattenedList += flattenLists(kvPairList)     #Flatten into a list of tuples
  
for tup in flattenedList:                       #Build the dictionary from tuples
  dataDict_Table2["Safety_Restraint"]["DATA"][tup[0]] = tup[1]



In [143]:
#Parse out the Driver License Origin Data:

dataDict_Table2["Driver_License_State"]["DATA"] = {}
flattenedList = []
desc1 = None
desc2 = None

for idx, series in Tables2_driverLicense.iterrows(): 

  desc1 = series['DRIVER LICENSE STATE']
  desc2 = series['Unnamed: 3']

  kvPairList1 = getKVPairs(desc1)                 #Get list (K,V) tuples for DATA
  kvPairList2 = getKVPairs(desc2)

  flattenedList += flattenLists(kvPairList1)     #Flatten into a list of tuples
  flattenedList += flattenLists(kvPairList2)

for tup in flattenedList:                       #Build the dictionary from tuples
  dataDict_Table2["Driver_License_State"]["DATA"][tup[0]] = tup[1]


In [146]:
#Create a JSON object of the Table2 Data
json_Table2 = json.dumps(dataDict_Table2, indent = 4) 


In [152]:
#Extract the messy table data from the TABLES3 Sheet 
  #(Manual inspection used to determine the cut points for the DFs)
Tables3_personRelatedFactors = pd.read_excel(DD, 'Tables3', skiprows = 0, nrows=100,
                               header = 0, usecols= 'A')
Tables3_criticalEvent = (pd.read_excel(DD, 'Tables3', skiprows = 1, nrows=65, 
                           usecols= 'E'))

In [155]:
#Parse out the PersonRelatedFactors Numbering System from Tables3
dataDict_Table3 = defaultdict(dict)                  #Build the Data Dictionary
                                                      #For all Table 3

dataDict_Table3["Person_Related_Factors"]["DATA"] = {}  #Create Keys for PRF
prf_flattenedList = []

for idx, series in Tables3_personRelatedFactors.iterrows(): 

  prf_desc = series['PERSON-RELATED FACTORS']

  kvPairList = getKVPairs(prf_desc)                 #Get list (K,V) tuples for DATA
  
  prf_flattenedList += flattenLists(kvPairList)     #Flatten into a list of tuples
  
for tup in prf_flattenedList:                       #Build the dictionary from tuples
  dataDict_Table3["Person_Related_Factors"]["DATA"][tup[0]] = tup[1]




In [189]:
#Parse out the Critical Event Factor Numbering System from Tables3

dataDict_Table3["Critical_Event"]["DATA"] = {}  #Create Keys for PRF
ce_flattenedList = []

#print(Tables3_criticalEvent.columns)

for idx, series in Tables3_criticalEvent.iterrows(): 

  ce_desc = series['THIS VEHICLE LOSS OF CONTROL DUE TO: ']

  kvPairList = getKVPairsSpaceDelimited(ce_desc)
  #print(kvPairList)

  ce_flattenedList += flattenLists(kvPairList)

for tup in ce_flattenedList:                       #Build the dictionary from tuples
  dataDict_Table3["Critical_Event"]["DATA"][tup[0]] = tup[1]  

  pass
  #print (series)


In [192]:
#Create a JSON object of the Table2 Data
json_Table3 = json.dumps(dataDict_Table3, indent = 4) 


In [194]:
#Extract the messy table data from the TABLES4 Sheet 
  #(Manual inspection used to determine the cut points for the DFs)
Tables4 = pd.read_excel(DD, 'Tables4')

Tables4

,CITY (Codes),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,10 ABERDEEN\n12 ACME\n15 ADDY\n17 ADNA\n19...,270 CASHMERE\n280 CASTLE ROCK \n290 CATHL...,545 EASTON\n555 EASTSOUND\n570 EAST WENATCH...,878 HADLOCK\n880 HAMILTON\n885 HANFORD\n887...,1192 LOOMIS\n1193 LOWDEN\n1194 LUMMI ISLAND...,1510 NORTH BONNEVILLE\n1512 NORTH \n1513 NO...,1854 RICHMOND\n HIGHLANDS\n1855 RICH...,2200 SUMAS\n2210 SUMNER\n2220 SUNNYSIDE\n22...,2562 WITHROW\n2565 WOODINVILLE\n2570 WOODLA...
1,135 BICKLETON\n140 BINGEN\n145 BIRCH BAY\n1...,460 COULEE \n465 COULEE DAM\n470 COUPEVILLE...,767 FORT WORDEN\n768 FOUR LAKES\n769 FOX IS...,1044 KEWA\n1045 KEYPORT\n1048 KINGSTON\n104...,1321 METHOW\n1322 MICA\n1336 MILAN\n1338 M...,1680 PE ELL\n1681 PINE \n1684 PLAZA\n1683 ...,2013 SILVER CREEK\n2012 SILVERDALE\n2015 SI...,2373 VASHON\n2374 VAUGHN\n2375 VERADALE\n238...,NaN
2,,NaN,NaN,\n\n\n \n,\n\n \n,\n,NaN,NaN,NaN
3,\n \n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,\,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [215]:
#Parse out the City Numbering System from Tables4
dataDict_Table4 = defaultdict(dict)                  #Build the Data Dictionary
                                                      #For all Table 4
dataDict_Table4["City"]["DATA"] = {}  #Create Keys for PRF
city_flattenedList = []

for idx, series in Tables4.iterrows(): 

  city_desc1 = series['CITY (Codes)']
  city_desc2 = series['Unnamed: 1']
  city_desc3 = series['Unnamed: 2']
  city_desc4 = series['Unnamed: 3']
  city_desc5 = series['Unnamed: 4']
  city_desc6 = series['Unnamed: 5']
  city_desc7 = series['Unnamed: 6']
  city_desc8 = series['Unnamed: 7']
  city_desc9 = series['Unnamed: 8']

  kvPairList1 = getKVPairsSpaceDelimited(city_desc1)                 #Get list (K,V) tuples for DATA
  kvPairList2 = getKVPairsSpaceDelimited(city_desc2)
  kvPairList3 = getKVPairsSpaceDelimited(city_desc3)
  kvPairList4 = getKVPairsSpaceDelimited(city_desc4)
  kvPairList5 = getKVPairsSpaceDelimited(city_desc5)
  kvPairList6 = getKVPairsSpaceDelimited(city_desc6)
  kvPairList7 = getKVPairsSpaceDelimited(city_desc7)
  kvPairList8 = getKVPairsSpaceDelimited(city_desc8)
  kvPairList9 = getKVPairsSpaceDelimited(city_desc9)


  city_flattenedList += flattenLists(kvPairList1)     #Flatten into a list of tuples
  city_flattenedList += flattenLists(kvPairList2)
  city_flattenedList += flattenLists(kvPairList3)
  city_flattenedList += flattenLists(kvPairList4)
  city_flattenedList += flattenLists(kvPairList5)
  city_flattenedList += flattenLists(kvPairList6)
  city_flattenedList += flattenLists(kvPairList7)
  city_flattenedList += flattenLists(kvPairList8)
  city_flattenedList += flattenLists(kvPairList9)

for tup in city_flattenedList:                       #Build the dictionary from tuples
  dataDict_Table4["City"]["DATA"][tup[0]] = tup[1]
  


In [217]:
json_Table4 = json.dumps(dataDict_Table4, indent = 4) 


In [228]:
#Extract the messy table data from the TABLES5 Sheet 
  #(Manual inspection used to determine the cut points for the DFs)
Tables5 = pd.read_excel(DD, 'Tables5')

Tables5

,Vehicle MAKE,Unnamed: 1
0,1=American Motors\n2=Jeep/Kaiser-Jeep/\n W...,66=Mahindra (Since 2011)\n67=Scion (Since 2012...
1,39=Jaguar\n40=Lancia\n41=Mazda\n42=Mercedes-Be...,93 MCI\n94 Thomas Built\n97 Not Reported (Sinc...


In [231]:
#Parse out the Vehicle Make Numbering System from Tables5
dataDict_Table5 = defaultdict(dict)                  #Build the Data Dictionary
                                                      #For all Table 3

print(Tables5.columns)                                                      

dataDict_Table5["Vehicle_Make"]["DATA"] = {}  #Create Keys for VM
vm_flattenedList = []

for idx, series in Tables5.iterrows(): 

  vm_desc1 = series['Vehicle MAKE']
  vm_desc2 = series['Unnamed: 1']

  kvPairList1 = getKVPairs(vm_desc1)                 #Get list (K,V) tuples for DATA
  kvPairList2 = getKVPairs(vm_desc2)

  vm_flattenedList += flattenLists(kvPairList1)     #Flatten into a list of tuples
  vm_flattenedList += flattenLists(kvPairList2)

for tup in vm_flattenedList:                       #Build the dictionary from tuples
  dataDict_Table5["Vehicle_Make"]["DATA"][tup[0]] = tup[1]

Index(['Vehicle MAKE', 'Unnamed: 1'], dtype='object')


In [233]:
json_Table5 = json.dumps(dataDict_Table5, indent = 4) 


In [234]:
#Extract the messy table data from the TABLES6 Sheet 
  #(Manual inspection used to determine the cut points for the DFs)
Tables6 = pd.read_excel(DD, 'Tables6')

Tables6

,CRASH TYPE (From 2010 on),Unnamed: 1,Unnamed: 2
0,0=No Impact \nCATEGORY I: SINGLE DRIVER \nCON...,39=This Vehicle Impacted by Frontal Area of An...,70=Initial Same Directions (Turning Right) \n7...
1,"\nCATEGORY II: SAME TRAFFICWAY, SAME DIRECTIO...",\nCONFIGURATION H: FORWARD IMPACT \n54=This V...,CATEGORY V: INTERSECTING PATHS (VEHICLE DAMAG...
2,\n\n,NaN,NaN


In [236]:
#Parse out the Crash Type Numbering System from Tables6
dataDict_Table6 = defaultdict(dict)                  #Build the Data Dictionary
                                                      #For all Table 3

print(Tables6.columns)                                                      

dataDict_Table6["Crash_Type"]["DATA"] = {}  #Create Keys for VM
ct_flattenedList = []

for idx, series in Tables6.iterrows(): 

  ct_desc1 = series['CRASH TYPE  (From 2010 on)']
  ct_desc2 = series['Unnamed: 1']
  ct_desc3 = series['Unnamed: 2']

  kvPairList1 = getKVPairs(ct_desc1)                 #Get list (K,V) tuples for DATA
  kvPairList2 = getKVPairs(ct_desc2)
  kvPairList2 = getKVPairs(ct_desc3)

  ct_flattenedList += flattenLists(kvPairList1)     #Flatten into a list of tuples
  ct_flattenedList += flattenLists(kvPairList2)
  ct_flattenedList += flattenLists(kvPairList3)

for tup in ct_flattenedList:                       #Build the dictionary from tuples
  dataDict_Table6["Crash_Type"]["DATA"][tup[0]] = tup[1]

Index(['CRASH TYPE  (From 2010 on)', 'Unnamed: 1', 'Unnamed: 2'], dtype='object')


In [239]:
json_Table6 = json.dumps(dataDict_Table6, indent = 4) 

In [263]:
#Extract the messy table data from the TABLES7 Sheet 
  #(Manual inspection used to determine the cut points for the DFs)
Tables7_driverRelatedFactors = pd.read_excel(DD, 'Tables7', skiprows = 0, nrows=6,
                               header = 0, usecols= 'A:C')

Tables7_nonMotoristLocation = pd.read_excel(DD, 'Tables7', skiprows = 0, nrows=3,
                               header = 0, usecols= 'D')

Tables7_nonMotoristLocation


,NON-MOTORIST LOCATION
0,"0=NOT a Non-Motorist (e.g., an MV occupant)\n1..."
1,20=(since 2010) Shoulder/Roadside\n21=(since 2...
2,NaN


In [261]:
#Parse out the Driver Related Factors Numbering System from Tables7
dataDict_Table7 = defaultdict(dict)                  #Build the Data Dictionary
                                                      #For all Table 7

print(Tables7_driverRelatedFactors.columns)                                                      

dataDict_Table7["Driver_Related_Factors"]["DATA"] = {}  #Create Keys for VM
drf_flattenedList = []

for idx, series in Tables7_driverRelatedFactors.iterrows(): 

  drf_desc1 = series['DRIVER-RELATED FACTORS']
  drf_desc2 = series['Unnamed: 1']
  drf_desc3 = series['Unnamed: 2']

  kvPairList1 = getKVPairs(drf_desc1)                 #Get list (K,V) tuples for DATA
  kvPairList2 = getKVPairs(drf_desc2)
  kvPairList2 = getKVPairs(drf_desc3)

  drf_flattenedList += flattenLists(kvPairList1)     #Flatten into a list of tuples
  drf_flattenedList += flattenLists(kvPairList2)
  drf_flattenedList += flattenLists(kvPairList3)

for tup in drf_flattenedList:                       #Build the dictionary from tuples
  dataDict_Table7["Driver_Related_Factors"]["DATA"][tup[0]] = tup[1]

Index(['DRIVER-RELATED FACTORS', 'Unnamed: 1', 'Unnamed: 2'], dtype='object')


In [264]:
#Parse out the Driver Related Factors Numbering System from Tables7

print(Tables7_nonMotoristLocation.columns)                                                      

dataDict_Table7["Non_Motorist_Location"]["DATA"] = {}  #Create Keys for VM
nml_flattenedList = []

for idx, series in Tables7_nonMotoristLocation.iterrows(): 

  nml_desc1 = series['NON-MOTORIST LOCATION']


  kvPairList1 = getKVPairs(nml_desc1)                 #Get list (K,V) tuples for DATA

  nml_flattenedList += flattenLists(kvPairList1)     #Flatten into a list of tuples


for tup in nml_flattenedList:                       #Build the dictionary from tuples
  dataDict_Table7["Non_Motorist_Location"]["DATA"][tup[0]] = tup[1]

Index(['NON-MOTORIST LOCATION'], dtype='object')


In [266]:
json_Table7 = json.dumps(dataDict_Table7, indent = 4) 


In [365]:
#Extract the messy table data from the TABLES8 Sheet 
  #(Manual inspection used to determine the cut points for the DFs)
Tables8_vehicleBodyType = pd.read_excel(DD, 'Tables8', skiprows = 0, nrows=4,
                               header = 0, usecols= 'A,B')

Tables8_vehicleFactors = pd.read_excel(DD, 'Tables8', skiprows = 0, nrows=4,
                               header = 0, usecols= 'C')

Tables8_vehicleManeuver = pd.read_excel(DD, 'Tables8', skiprows = 0, nrows=4,
                               header = 0, usecols= 'D')

Tables8_crashRelatedFactors = pd.read_excel(DD, 'Tables8', skiprows = 0, nrows=3,
                               header = 0, usecols= 'E')


Tables8_crashRelatedFactors

,Crash Related Factors
0,"00=None\n01=Inadequate Warning of Exits, Lanes..."
1,"14=Motor Vehicle Struck by Falling Cargo, or S..."
2,NaN


In [368]:
#Parse out the Vehicle Body Type Numbering System from Tables8

dataDict_Table8 = defaultdict(dict)

print(Tables8_vehicleBodyType.columns)                                                      

dataDict_Table8["Vehicle_Body_Type"]["DATA"] = {}  #Create Keys for VM
ln_vbt_desc1 = []
ln_vbt_desc2 = []

for idx, series in Tables8_vehicleBodyType.iterrows(): 

  vbt_desc1 = series['VEHICLE BODY TYPE']
  vbt_desc2 = series['Unnamed: 1']
                                              #Split the cell into lines
  if type(vbt_desc1) != float:
    ln_vbt_desc1 += vbt_desc1.splitlines()  
  else:
    pass
  
  if type(vbt_desc2) != float:
    ln_vbt_desc2 += vbt_desc2.splitlines()
  else:
    pass

                                              #Get the KV Pairs & Update Dict
kvPairs1 = []
for ln in ln_vbt_desc1:
  if ln == "" or ln == " " or ln == None:
    continue
  else:
    if ln[0].isnumeric():
      p2 = ln.split("=")
      dataDict_Table8["Vehicle_Body_Type"]["DATA"][p2[0]] = p2[1]
    else:
      pass

kvPairs2 = []
for ln in ln_vbt_desc2:
  
  if ln == "" or ln == " " or ln == None:
    continue
  else:
    if ln[0].isnumeric():
      p2 = ln.split("=")
      dataDict_Table8["Vehicle_Body_Type"]["DATA"][p2[0]] = p2[1]
    else:
      pass
else:
  pass


Index(['VEHICLE BODY TYPE', 'Unnamed: 1'], dtype='object')


In [372]:
#Parse out the Vehicle Factors Numbering System from Tables8

print(Tables8_vehicleFactors.columns)                                                      

dataDict_Table8["Vehicle_Factors"]["DATA"] = {}  #Create Keys for VM
ln_vf_desc1 = []

for idx, series in Tables8_vehicleFactors.iterrows(): 

  vf_desc1 = series['VEHICLE FACTORS 1 & 2']

  if type(vf_desc1) != float:
    ln_vf_desc1 += vf_desc1.splitlines()  
  else:
    pass
  
kvPairs1 = []
for ln in ln_vf_desc1:
  if ln == "" or ln == " " or ln == None:
    continue
  else:
    if ln[0].isnumeric():
      p2 = ln.split("=")
      dataDict_Table8["Vehicle_Factors"]["DATA"][p2[0]] = p2[1]
    else:
      pass



Index(['VEHICLE FACTORS 1 & 2'], dtype='object')


In [374]:
#Parse out the Vehicle Maneuver Numbering System from Tables8

print(Tables8_vehicleManeuver.columns)                                                      

dataDict_Table8["Vehicle_Maneuver"]["DATA"] = {}  #Create Keys for VM
ln_vm_desc1 = []

for idx, series in Tables8_vehicleManeuver.iterrows(): 

  vm_desc1 = series['VEHICLE MANEUVER']

  if type(vm_desc1) != float:
    ln_vm_desc1 += vm_desc1.splitlines()  
  else:
    pass
  
kvPairs1 = []
for ln in ln_vm_desc1:
  if ln == "" or ln == " " or ln == None:
    continue
  else:
    if ln[0].isnumeric():
      p2 = ln.split("=")
      dataDict_Table8["Vehicle_Maneuver"]["DATA"][p2[0]] = p2[1]
    else:
      pass

Index(['VEHICLE MANEUVER'], dtype='object')


In [376]:
#Parse out the Vehicle Crash Related Factors Numbering System from Tables8

print(Tables8_crashRelatedFactors.columns)                                                      

dataDict_Table8["Vehicle_Crash_Related_Factors"]["DATA"] = {}  #Create Keys for VM
ln_vcrf_desc1 = []

for idx, series in Tables8_crashRelatedFactors.iterrows(): 

  vcrf_desc1 = series['Crash Related Factors']

  if type(vcrf_desc1) != float:
    ln_vcrf_desc1 += vcrf_desc1.splitlines()  
  else:
    pass
  
kvPairs1 = []
for ln in ln_vcrf_desc1:
  if ln == "" or ln == " " or ln == None:
    continue
  else:
    if ln[0].isnumeric():
      p2 = ln.split("=")
      dataDict_Table8["Vehicle_Crash_Related_Factors"]["DATA"][p2[0]] = p2[1]
    else:
      pass

Index(['Crash Related Factors'], dtype='object')


In [378]:
json_Table8 = json.dumps(dataDict_Table8, indent = 4) 


In [395]:
#Extract the messy table data from the TABLES8 Sheet 
  #(Manual inspection used to determine the cut points for the DFs)
Tables9 = pd.read_excel(DD, 'Tables9', skiprows = 2, nrows=56,
                               header = None, usecols= 'B:H')

s1 = pd.Series(Tables9[1])    #Break out each series and join them into a singl
s2 = pd.Series(Tables9[2])      #Column in the T9 DataFrame
s3 = pd.Series(Tables9[3])
s4 = pd.Series(Tables9[4])
s5 = pd.Series(Tables9[5])
s6 = pd.Series(Tables9[6])
s7 = pd.Series(Tables9[7])

T9 = pd.DataFrame(pd.concat([s1,s2,s3,s4,s5,s6,s7], ignore_index=True))


,0
0,(1)WSP-3
1,(2)WSP-7
2,(3)WSP-1
3,(4)WSP-4
4,(5)WSP-5
...,...
373,(344)Yakama Nation PD
374,(345)Spokane Tribal PD
375,(346)Liberty Lake PD
376,(347) Shoalwater Bay Tribal PD


In [403]:
#Parse out the Vehicle Crash Related Factors Numbering System from Tables8

dataDict_Table9 = defaultdict(dict)

print(T9.columns)                                                      

dataDict_Table9["Reporting_Jurisdiction"]["DATA"] = {}  #Create Keys for VM
ln_rj_desc1 = []

for idx, series in T9.iterrows(): 

  rj_desc1 = series[0]

  if type(rj_desc1) != float:
    ln_rj_desc1 += rj_desc1.splitlines()  
  else:
    pass

print(ln_rj_desc1)

for ln in ln_rj_desc1:
  LHS, RHS = ln.split(")")
  LHS = LHS.strip("(")
  dataDict_Table9["Reporting_Jurisdiction"]["DATA"][LHS] = RHS


RangeIndex(start=0, stop=1, step=1)
['(1)WSP-3', '(2)WSP-7', '(3)WSP-1', '(4)WSP-4', '(5)WSP-5', '(6)WSP-6', '(7)WSP-2', '(8)WSP-8', '(9)Adams County', '(10)Asotin County', '(11)Benton County', '(12)Chelan County', '(13)Clallam County', '(14)Clark County', '(15)Columbia County', '(16)Cowlitz County', '(17)Douglas County', '(18)Ferry County', '(19)Franklin County', '(20)Garfield County', '(21)Grant County', '(22)Grays Harbor County', '(23)Island County', '(24)Jefferson County', '(25)King Co Dpt Pub Safe', '(26)Kitsap County', '(27)Kittitas County', '(28)Klickitat County', '(29)Lewis County', '(30)Lincoln County', '(31)Mason County', '(32)Okanogan County', '(33)Pacific County', '(34)Pend Oreille County', '(35)Pierce County', '(36)San Juan County', '(37)Skagit County', '(38)Skamania County', '(39)Snohomish County', '(40)Spokane County', '(41)Stevens County', '(42)Thurston County', '(43)Wahkiakum County', '(44)Walla Walla County', '(45)Whatcom County', '(46)Whitman County', '(47)Aberdeen P

In [404]:
json_Table9 = json.dumps(dataDict_Table9, indent = 4) 

print(json_Table9)


{
    "Reporting_Jurisdiction": {
        "DATA": {
            "1": "WSP-3",
            "2": "WSP-7",
            "3": "WSP-1",
            "4": "WSP-4",
            "5": "WSP-5",
            "6": "WSP-6",
            "7": "WSP-2",
            "8": "WSP-8",
            "9": "Adams County",
            "10": "Asotin County",
            "11": "Benton County",
            "12": "Chelan County",
            "13": "Clallam County",
            "14": "Clark County",
            "15": "Columbia County",
            "16": "Cowlitz County",
            "17": "Douglas County",
            "18": "Ferry County",
            "19": "Franklin County",
            "20": "Garfield County",
            "21": "Grant County",
            "22": "Grays Harbor County",
            "23": "Island County",
            "24": "Jefferson County",
            "25": "King Co Dpt Pub Safe",
            "26": "Kitsap County",
            "27": "Kittitas County",
            "28": "Klickitat County",
            "29